In [1]:
import os
import numpy as np
caffe_root = '/home/flathead/bitbucket/mycaffe/'
os.chdir('../../')
import sys
sys.path.insert(0, 'python')
import caffe

In [2]:
from pylab import *
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
from numpy.random import normal

In [4]:
caffe.set_mode_gpu()
caffe.set_device(0)

In [5]:
solver = caffe.SGDSolver('examples/mnist/test/lenet_solver.prototxt')

In [6]:
train_net = solver.net
test_net = solver.test_nets[0]

In [7]:
# during training, each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

[('data', (64, 1, 28, 28)),
 ('label', (64,)),
 ('conv1', (64, 20, 24, 24)),
 ('pool1', (64, 20, 12, 12)),
 ('conv2', (64, 50, 8, 8)),
 ('pool2', (64, 50, 4, 4)),
 ('ip1', (64, 500)),
 ('ip2', (64, 10)),
 ('loss', ())]

In [8]:
# just print the weight sizes (we'll omit the biases)
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

[('conv1', (20, 1, 5, 5)),
 ('conv2', (50, 20, 5, 5)),
 ('ip1', (500, 800)),
 ('ip2', (10, 500))]

In [14]:
solver.net.blobs['ip2'].data[99]

IndexError: index 99 is out of bounds for axis 0 with size 64

In [9]:
%%time
niter = 10000
test_interval = 500
train_loss = zeros(niter)
test_acc = zeros(int(np.ceil(niter / test_interval)))
output = zeros((niter, 8, 10))

for it in range(niter):
    solver.step(1)
    
    train_loss[it] = train_net.blobs['loss'].data
    
    test_net.forward(start='conv1')
    output[it] = test_net.blobs['ip1'].data[:8]
    
    if it % test_interval == 0:
        print 'iteration', it, 'testing...'
        correct = 0
        for test_it in range(100):
            test_net.forward()
            correct += sum(test_net.blobs['ip1'].data.argmax(1) == test_net.blobs['label'].data)
        test_acc[it // test_interval] = correct / 1e4

ValueError: could not broadcast input array from shape (8,500) into shape (8,10)

In [ ]:
_, ax1 = subplots()
ax2 = ax1.twinx()
ax1.plot(arange(niter), train_loss)
ax2.plot(test_interval * arange(len(test_acc)), test_acc, 'r')
ax1.set_xlabel('iteration')
ax1.set_ylabel('train loss')
ax2.set_ylabel('test accuracy')
ax2.set_title('Test Accuracy: {:.2f}'.format(test_acc[-1]))

In [ ]:
plt.figure(figsize=(15, 15))

feat1 = train_net.params['conv1'][0].data
feat2 = train_net.params['conv2'][0].data
feat3 = train_net.params['conv3'][0].data
feat4 = train_net.params['ip1'][0].data

plt.subplot(2, 2, 1)
#plt.hist(train_net.params['ip1'][0].data)
plt.hist(feat1)
plt.title("ip1_weight")

#plt.xlabel("weight")
plt.subplot(2, 2, 2)
#plt.hist(train_net.params['ip2'][0].data)
plt.hist(feat2)
plt.title("ip2_weight")

plt.subplot(2, 2, 3)
#plt.hist(train_net.params['ip3'][0].data)
plt.hist(feat3)
plt.title("ip3_weight")

plt.subplot(2, 2, 4)
#plt.hist(train_net.params['ip4'][0].data)
plt.hist(feat4)
plt.title("ip4_weight")

#plt.subplot_tool()
plt.show()

In [ ]:
train_net.params['conv2'][0].data[0][1].flat

In [ ]:
train_net.params['conv3'][0].data[64]

In [ ]:
train_net.params['conv1'][0].data.shape

In [ ]:
train_net.blobs['conv1'].data.shape

In [ ]:
train_net.params['conv1'][0].data.shape